In [49]:
setwd("/home/simingz/run_rasqual/datarun/CN")

See header info [here](https://github.com/natsuhiko/rasqual/blob/master/README.md)

In [50]:
addgFDR <- function(mrestable, prestable){
    mrestable$geneFDR <- NA
    peakIDs <- unique(mrestable$V1)
    for (peakID in peakIDs) {
        tt <- max(mrestable[mrestable$V1==peakID,11])
        ttperm <- prestable[prestable$V1==peakID,3] # max test statistic for each permutation
        gfdr <- length(ttperm[ttperm > tt])/length(ttperm)
        mrestable[mrestable$V1==peakID,"geneFDR"]=gfdr
    }
    return(mrestable)
}

addgFDR_V2_getperm <- function(mrestable, prestable){
    peakIDs <- unique(mrestable$V1)
    out <- NULL
    for (peakID in peakIDs) {
        ntest <- mrestable[mrestable$V1==peakID,18][1]
        tt <- max(mrestable[mrestable$V1==peakID,11])
        preal <- pchisq(tt,df=1,lower.tail=F,log.p=F)
        ttperm <- prestable[prestable$V1==peakID,3][1] # keep only 1 permutation
        pperm <- pchisq(ttperm,df=1,lower.tail=F,log.p=F)
        out <-rbind(out,c(log(preal*ntest),log(pperm*ntest))) # bonferroni corrected p value
    }
    return(out)
}

In [ ]:
mainfall= Sys.glob( paste0("mainrun_chr*_CN*.result.txt"))
#outmres <- NULL
outlogp <- NULL
for (mainf in mainfall){
    permf = gsub("result.txt","permres.txt",gsub("main","perm",mainf)) 
    mres <- read.table(mainf,header=F,stringsAsFactors = F, sep="\t",fill=T)
    pres <- read.table(permf,header=F,stringsAsFactors = F, sep="\t",fill=T)
    mresfl <- mres[mres$V2 != "SKIPPED",]
    if (dim(mresfl)[1]==0){next}
    #outmres<- rbind(outmres, addgFDR(mresfl,pres)) # FDR method1
    outlogp <- rbind(outlogp, addgFDR_V2_getperm(mresfl,pres))# FDR method2
} 

In [ ]:
outmres$geneFDR2 <- NA
peakIDs <- unique(outmres$V1)
outreal <- outlogp[,1]
outperm <- outlogp[,2]
for (peakID in peakIDs){
    tt <- max(outmres[outmres$V1==peakID,11])
    p <- pchisq(tt,df=1,lower.tail=F,log.p=F)
    ntest <- outmres[outmres$V1==peakID,18][1]
    logpBon <- log(p*ntest)
    gfdr <- length(outperm[outperm < logpBon])/length(outreal[outreal < logpBon])
    outmres[outmres$V1==peakID,"geneFDR2"]=gfdr
}

In [ ]:
write.table( outmres , file= "CN_resultsFDR.txt" , row.names=F, col.names=T, sep="\t", quote = F)

In [60]:
length(unique(outmres[outmres$geneFDR2 < 0.1,]$V1))

[1] 104

ERROR: Error in xy.coords(x, y, xlabel, ylabel, log): 'x' and 'y' lengths differ


In [ ]:
length(unique(outmres$V1))

In [66]:
outmres[outmres$V1=="newCN100984",]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V18,V19,V20,V21,V22,V23,V24,V25,geneFDR,geneFDR2
160097,newCN100984,8:102307500,chr8,101295272,C,T,0.100,0.246914,0.805265,-0.1338522,⋯,10,6,6,101298248,-168.8818,0,NaN,0.660091,0.004,0.8663625
212838,newCN100984,8:102307591,chr8,101295363,T,G,0.100,0.246914,0.809241,-0.1338522,⋯,10,6,6,101298248,-168.8818,0,NaN,0.665447,0.004,0.8663625
312832,newCN100984,8:102307638,chr8,101295410,C,T,0.100,0.246914,0.808516,-0.1338522,⋯,10,6,6,101298248,-168.8818,0,NaN,0.664466,0.004,0.8663625
412822,newCN100984,8:102307870,chr8,101295642,C,A,0.475,0.211180,0.885436,-2.0159319,⋯,10,6,6,101298248,-168.8818,0,NaN,0.789191,0.004,0.8663625
512804,newCN100984,8:102308173,chr8,101295945,T,C,0.475,0.211180,0.884271,-2.0159319,⋯,10,6,6,101298248,-168.8818,0,NaN,0.787272,0.004,0.8663625
612807,newCN100984,8:102309013,chr8,101296785,T,C,0.475,0.211180,0.867622,-2.0159319,⋯,10,5,6,101298248,-168.8818,0,NaN,0.760296,0.004,0.8663625
712745,newCN100984,8:102310476,chr8,101298248,G,A,0.350,0.292235,0.839362,-2.0159319,⋯,10,7,6,101298248,-168.8818,0,NaN,0.761758,0.004,0.8663625
812092,newCN100984,8:102310694,chr8,101298466,C,T,0.100,0.246914,0.887388,-0.1338522,⋯,10,5,6,101298248,-168.8818,0,NaN,0.782455,0.004,0.8663625
910988,newCN100984,8:102310703,chr8,101298475,G,A,0.075,0.131483,0.758086,-0.1338522,⋯,10,6,6,101298248,-168.8818,0,NaN,0.614621,0.004,0.8663625
1010935,newCN100984,8:102310808,chr8,101298580,G,A,0.100,0.246914,0.885309,-0.1338522,⋯,10,5,6,101298248,-168.8818,0,NaN,0.779026,0.004,0.8663625


In [63]:
p <- pchisq(8.77,df=1,lower.tail=F,log.p=F)

In [67]:
log(p*10)

[1] -3.486019